# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [21]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade
from CNN_layers import linear_layer_homemade
from CNN_layers import elu_homemade
from CNN_layers import relu_homemade as ReLU
from Tests.helper_functions import tokenize, transform_input, compare, create_conv_homemade, \
                                create_batchnorm_homemade, create_maxpool_homemade

In [5]:
MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

In [6]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [7]:
input_original_test, label_test = next(iter(test_loader))
input_homemade_test = transform_input(input_batch=input_original_test)

In [8]:
elu_homemade = elu_homemade.ELU()
relu_homemade = ReLU.relu()
conv1_homemade = create_conv_homemade(model_conv= model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm1)
maxpool1_homemade = create_maxpool_homemade(model_maxpool= model_original.maxPool1)
conv2_homemade = create_conv_homemade(model_conv= model_original.conv2)
batchnorm2_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm2)
maxpool2_homemade = create_maxpool_homemade(model_original.maxPool2)
linear_homemade = linear_layer_homemade.linear_layer(model_original.lin.weight,model_original.lin.bias,2)

STRIDE ER PRÆDEFINERET
STRIDE ER PRÆDEFINERET


## Shape of the network architecture

In [9]:
print(model_original)

CNNSmall(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchNorm1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxPool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxPool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (lin): Linear(in_features=45, out_features=2, bias=True)
  (network): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(5, eps=1e-05, momentum=0.1, aff

## Testing the first convolutional layer

In [10]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

Convolutional layer 1 error over out channels: 


[[2.714658446367757e-06, 2.784962947907643e-06, 1.9446338829220977e-06],
 [5.921787107621146e-06, 4.750881298247467e-06, 3.3989394729316835e-06]]

## Testing the first Batchnorm layer

In [11]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

BatchNorm layer 1 error over out channels: 


[[4.330277442932129e-05, 2.0656734704971313e-05, 7.69440084695816e-05],
 [5.023926496505737e-05, 3.732740879058838e-05, 7.526762783527374e-05]]

## Testing the first ReLU layer

In [12]:
out_homemade = relu_homemade(out_homemade)

out_original = model_original.relu1(out_original)

print("ReLU layer 1 error over out channels")
compare(out_homemade, transform_input(input_batch = out_original))

ReLU layer 1 error over out channels


[[8.52346420288086e-06, 1.2625008821487427e-05, 6.531365215778351e-05],
 [1.8537044525146484e-05, 1.92299485206604e-05, 5.605258047580719e-05]]

## Testing the first Maxpool layer

In [13]:
# homemade MaxPool filter on test
out_homemade = maxpool1_homemade(input_batch=out_homemade)
# original MaxPool filter on test
out_original = model_original.maxPool1(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[3.3602118492126465e-06, 6.295740604400635e-06, 1.7449259757995605e-05],
 [7.860362529754639e-06, 8.106231689453125e-06, 1.5363097190856934e-05]]

## Testing the second Convolutional layer

In [14]:
# TODO: test the second convolutional layer

# homemade batchNorm1 filter on test
out_homemade = conv2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.conv2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[7.265901361658278e-06,
  6.0008974384351266e-06,
  6.8729584533633314e-06,
  8.080163430713994e-06,
  6.1661622124398185e-06],
 [9.889932549300706e-06,
  8.173251276083482e-06,
  8.50474784619415e-06,
  1.0051189772125602e-05,
  7.640689441951176e-06]]

## Testing the second Batchnorm layer

In [15]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[2.3447908461093903e-05,
  2.220645546913147e-05,
  2.6049092411994934e-05,
  2.2366642951965332e-05,
  1.528114080429077e-05],
 [3.1970441341400146e-05,
  3.1556934118270874e-05,
  2.845749258995056e-05,
  2.7514994144439697e-05,
  1.93864107131958e-05]]

## Testing the second Maxpool layer

In [16]:
# homemade batchNorm1 filter on test
out_homemade = maxpool2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.maxPool2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


[[3.777444362640381e-06,
  2.9802322387695312e-06,
  3.341585397720337e-06,
  1.9669532775878906e-06,
  2.376735210418701e-06],
 [4.76837158203125e-06,
  3.129243850708008e-06,
  3.933906555175781e-06,
  1.6093254089355469e-06,
  2.816319465637207e-06]]

## Testing the Linear layer

In [17]:
# homemade linear filter on test
out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
# original linear filter on test
out_original = model_original.lin(torch.reshape(out_original, (2,45)))

print("Linear layer 1 error over out channels: ")
compare(out_homemade.numpy(), list(list(out_original.detach().numpy())))

Linear layer 1 error over out channels: 


C:\Users\sture\AppData\Local\Temp\ipykernel_18508\3903311325.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
C:\Users\sture\Documents\Studie\Bachelor\bachelorprojekt\Python\CNN_layers\linear_layer_homemade.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp = torch.dot(input[n],torch.tensor(self.weights[i], dtype = torch.double)) + torch.tensor(self.bias[i], dtype = torch.double)
C:\Users\sture\Documents\Studie\Bachelor\bachelorprojekt\Python\CNN_layers\linear_layer_

[[7.292877981512902e-07, 6.724754975095948e-07],
 [5.000813505517954e-07, 4.0577709903288905e-07]]

In [18]:
import torch.nn as nn

In [30]:
relu = nn.ReLU()
homemade = ReLU.relu()

print(relu(torch.tensor([0,1,2,-3,-0.32543254,5,6,7])))
print(homemade([[[[0,1,2,-3,0.546547,5,6,7]]]]))

tensor([0., 1., 2., 0., 0., 5., 6., 7.])
[[[[0.       1.       2.       0.       0.546547 5.       6.
    7.      ]]]]
